In [1]:
%%capture
# !unzip '/kaggle/input/street-view-getting-started-with-julia/train.zip'
# !unzip '/kaggle/input/street-view-getting-started-with-julia/test.zip'

!unzip '/kaggle/input/street-view-getting-started-with-julia/trainResized.zip'
!unzip '/kaggle/input/street-view-getting-started-with-julia/testResized.zip'

In [2]:
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1,L2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D,Dense,BatchNormalization,Dropout,Flatten,MaxPool2D


from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import os
from PIL import Image
from skimage.io import imread

def read_image(folder_path):
    image_list = []
    files = os.listdir(folder_path)
    files = sorted(files, key=lambda x: int(x[:-4]))
    for filename in files:
        file_path = os.path.join(folder_path, filename)
        image = imread(file_path)
        if len(image.shape)==2:
            image=np.repeat(image[:, :, np.newaxis], 3, axis=2)
        image_list.append(image)
    return image_list

train_image=read_image('/kaggle/working/trainResized')
test_image=read_image('/kaggle/working/testResized')

# train_image=read_image('/kaggle/working/train')
# test_image=read_image('/kaggle/working/test')
print('Number of train image: ',len(train_image))
print('Number of test image: ',len(test_image))

Number of train image:  6283
Number of test image:  6220


In [4]:
from skimage.transform import resize

new_shape=(20,20)
# using pillow
# train_image=[image.resize(new_shape) for image in train_image]
# test_image=[image.resize(new_shape) for image in test_image]
# using skimage
# train_image=[resize(image,new_shape) for image in train_image]
# test_image=[resize(image,new_shape) for image in test_image]

train_image=np.asarray(train_image,dtype=np.float32)
test_image=np.asarray(test_image,dtype=np.float32)

In [5]:
# data augmentaion
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator=ImageDataGenerator(
    zoom_range = 0.2,
    rescale = 1./255,
    rotation_range = 5.0,
    shear_range = 3.0,
    brightness_range = [0.0, 3.0]
)

In [6]:
label=pd.read_csv('/kaggle/input/street-view-getting-started-with-julia/trainLabels.csv')
unique_label=np.sort(label['Class'].unique())
print('Number of label: ',len(unique_label))
label_to_id={}
for i in range(len(unique_label)):
    label_to_id[unique_label[i]]=i

ids_lable=np.asarray([label_to_id[i] for i in label['Class'].values],dtype=np.float32)
train_image,val_image,train_label,val_label=train_test_split(train_image,ids_lable,train_size=0.9,shuffle=True)

Number of label:  62


In [7]:
model = tf.keras.models.Sequential([
    BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3),padding='same', activation = "gelu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(64, (3,3),padding='same', activation = "gelu"),
    tf.keras.layers.MaxPool2D(2,2),
    BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    
    tf.keras.layers.Conv2D(128, (3,3),padding='same', activation = "gelu"),
#     tf.keras.layers.MaxPool2D(2,1),
    tf.keras.layers.Dropout(0.1),
    
    tf.keras.layers.Conv2D(128, (3,3),padding='same', activation = "gelu"),
    tf.keras.layers.MaxPool2D(2,2),
    
    BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(432, activation = "gelu"),
    BatchNormalization(),
    Dropout(0.2),
    tf.keras.layers.Dense(62, activation='softmax')
])

In [8]:
early_stopping=EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=15,
    restore_best_weights=True)

lr_schedule=ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100,
    decay_rate=0.9,
    staircase=True
)
lr_reduce=ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    min_delta=0.001,
    patience=10,
    min_lr=1e-8,
    verbose=1)

In [9]:
# training
optimizer=Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.98,epsilon=1e-9)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(data_generator.flow(train_image,train_label,batch_size=32),
        validation_data=(val_image,val_label),
         callbacks=[early_stopping,lr_reduce],
         epochs=75)

Epoch 1/75


2023-12-19 13:53:13.142696: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


177/177 [==============================] - 12s 30ms/step - loss: 4.1721 - accuracy: 0.1021 - val_loss: 129.6013 - val_accuracy: 0.0302 - lr: 9.0000e-04
Epoch 2/75
177/177 [==============================] - 4s 25ms/step - loss: 2.5983 - accuracy: 0.3773 - val_loss: 229.0060 - val_accuracy: 0.1049 - lr: 7.2900e-04
Epoch 3/75
177/177 [==============================] - 4s 25ms/step - loss: 1.8431 - accuracy: 0.5302 - val_loss: 126.3966 - val_accuracy: 0.1701 - lr: 5.9049e-04
Epoch 4/75
177/177 [==============================] - 4s 25ms/step - loss: 1.5036 - accuracy: 0.6130 - val_loss: 67.2948 - val_accuracy: 0.3323 - lr: 4.7830e-04
Epoch 5/75
177/177 [==============================] - 4s 25ms/step - loss: 1.3186 - accuracy: 0.6454 - val_loss: 35.6977 - val_accuracy: 0.4626 - lr: 4.3047e-04
Epoch 6/75
177/177 [==============================] - 4s 25ms/step - loss: 1.1856 - accuracy: 0.6836 - val_loss: 37.4112 - val_accuracy: 0.3911 - lr: 3.4868e-04
Epoch 7/75
177/177 [=====================

In [10]:
# training
optimizer=Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.98,epsilon=1e-9)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(train_image,train_label,batch_size=256,
        validation_data=(val_image,val_label),
         callbacks=[early_stopping,lr_reduce],
         epochs=75)

Epoch 1/75


2023-12-19 13:55:17.614877: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


23/23 [==============================] - 6s 51ms/step - loss: 0.4442 - accuracy: 0.8723 - val_loss: 1.2119 - val_accuracy: 0.6884 - lr: 0.0010
Epoch 2/75
23/23 [==============================] - 1s 32ms/step - loss: 0.2898 - accuracy: 0.9165 - val_loss: 1.0321 - val_accuracy: 0.7409 - lr: 0.0010
Epoch 3/75
23/23 [==============================] - 1s 32ms/step - loss: 0.2097 - accuracy: 0.9427 - val_loss: 0.9694 - val_accuracy: 0.7520 - lr: 0.0010
Epoch 4/75
23/23 [==============================] - 1s 33ms/step - loss: 0.1581 - accuracy: 0.9591 - val_loss: 0.9489 - val_accuracy: 0.7599 - lr: 0.0010
Epoch 5/75
23/23 [==============================] - 1s 32ms/step - loss: 0.1180 - accuracy: 0.9719 - val_loss: 0.9573 - val_accuracy: 0.7456 - lr: 9.0000e-04
Epoch 6/75
23/23 [==============================] - 1s 32ms/step - loss: 0.1057 - accuracy: 0.9719 - val_loss: 0.9589 - val_accuracy: 0.7583 - lr: 9.0000e-04
Epoch 7/75
23/23 [==============================] - 1s 32ms/step - loss: 0.0934

In [11]:
#write output to csv
pred=model.predict(test_image)
pred_label=list(map(lambda x: unique_label[x],np.argmax(pred,axis=1)))
submission=pd.read_csv('/kaggle/input/street-view-getting-started-with-julia/sampleSubmission.csv')

submission['Class']=pred_label
submission.to_csv('submission.csv',index=False)

195/195 [==============================] - 1s 3ms/step
